# Web Scraping with BeautifulSoup and Writing Data

In this notebook, I've been given a task to do a **Digimon List** data scraping from this [digimon website](http://digidb.io/digimon-list/), transfering the scraped data into these type of files **(.CSV, .JSON, .XLSX)** and create a databases **(MongoDB and MySQL)** using the libraries that I've learned. 

The tasks will be broken down into:  
1. Scraping data from the website.
2. Saving the scraped data into .JSON format.
3. Saving the scraped data into .CSV format.
4. Saving the scraped data into .XLSX format.
5. Creating MySQL databases and storing the scraped data.
6. Creating MongoDB databases and storing the scraped data.  

Okay, let's get started!

-----

### 1: Scraping Data

In this process, I want to scrape the data and transform those data into a list of dictionaries. Why? Because in list of dictionaries format, the next process (saving data into .JSON, .XLSX, .CSV format) will be so much easier. So, the goal in this process will be extracting data from website and transforming it into list of dictionaries.

To scrape data from the [digimon website](http://digidb.io/digimon-list/) in Python, we can use these 2 libraries: [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) and [requests](https://requests.readthedocs.io/en/master/). So let's import those 2 libraries (if you haven't install the libraries, please install it first).

In [10]:
from bs4 import BeautifulSoup
import requests

Next, I'm gonna specify the url we want to scrape and use *requests* to access the url and test if the url connected or not.

In [11]:
url = "http://digidb.io/digimon-list/"
html = requests.get(url)

if html.status_code == 200:
    print("Connected!")
else:
    print(html.status_code)

Connected!


Then, I create the *soup* object to use the BeautifulSoup class, and passing the parameter needed in the object.

In [12]:
soup = BeautifulSoup(html.content, 'html.parser')

To get the table heading, I can do this:

In [13]:
heading = []

for item in soup.find_all('th'):
    heading.append(item.text)
    
print(heading)

['#', 'Digimon', 'Stage', 'Type', 'Attribute', 'Memory', 'Equip Slots', 'HP', 'SP', 'Atk', 'Def', 'Int', 'Spd']


Notice that the table heading above doesn't include the image, and I want to add image column after the '#' column to hold the image link data.  
So, I'm gonna do the following syntax to create the 'Image Link' column:

In [14]:
heading.insert(1, 'Image Link')

print(heading)

['#', 'Image Link', 'Digimon', 'Stage', 'Type', 'Attribute', 'Memory', 'Equip Slots', 'HP', 'SP', 'Atk', 'Def', 'Int', 'Spd']


Okay, now I've got all the column header.  
Next, I'm gonna scrape the data in the table. In this process, I'm splitting it into two syntax, the first one to get table data and the second one to get the image url.  

**Step 1:**   
Get the table data and place it in a list variable called **_rawData_**.

In [15]:
rawData = []
for item in soup.find_all('td'):
    if item.text.strip().isdigit(): #to check if the item is number or not, if true cast it into int else let it be
        rawData.append(int(item.text.strip()))
    else:
        rawData.append(item.text.strip())
        
print(f"This is the preview of the first row data: \n{rawData[0:13]}")

#rawData[0:13] are data on the first row after the column title. 
#Why [0:13]? Because every row consist of 13 column. If you want to grab the second row, you can grab the [13:26], and so on.

This is the preview of the first row data: 
[1, 'Kuramon', 'Baby', 'Free', 'Neutral', 2, 0, 590, 77, 79, 69, 68, 95]


**Step 2:**  
Get the image url data and place it in a list called **_img_**.

In [16]:
img = []
for item in soup.find_all('img', style="vertical-align:middle;"):
    img.append(item['src'])
    
print(f"This is the preview of the first row image url data: \n{img[0]}")

This is the preview of the first row image url data: 
http://digidb.io/images/dot/dot629.png


After I get the table data an image url data, the next step I wanna do is to create a dictionary and append it into a list called **_processedData_** to create list of dictionary. The dictionary will be using each of the element in **_heading_** as a key, and for the value it'll be using both of the **_img_** and **_rawData_** elements.  

In [17]:
processedData = [] #variable to keep the dictionary I want to process
indexImg = 0 #index counter to keep track of data in img
indexRawData = 0 #index counter to keep track of data in rawData
addition = 13 #variable used to navigate row in rawData 

for item in rawData:
    while indexRawData < len(rawData):
        processedData.append({heading[0]:rawData[indexRawData], heading[1]:img[indexImg], heading[2]:rawData[indexRawData+1], heading[3]:rawData[indexRawData+2], 
                              heading[4]:rawData[indexRawData+3], heading[5]:rawData[indexRawData+4], heading[6]:rawData[indexRawData+5], heading[7]:rawData[indexRawData+6], 
                              heading[8]:rawData[indexRawData+7], heading[9]:rawData[indexRawData+8], heading[10]:rawData[indexRawData+9], heading[11]:rawData[indexRawData+10], 
                              heading[12]:rawData[indexRawData+11], heading[13]:rawData[indexRawData+12]})
        indexRawData += addition
        indexImg += 1
        
print(f"This is a preview of the first index in processedData: \n{processedData[0]}")

This is a preview of the first index in processedData: 
{'#': 1, 'Image Link': 'http://digidb.io/images/dot/dot629.png', 'Digimon': 'Kuramon', 'Stage': 'Baby', 'Type': 'Free', 'Attribute': 'Neutral', 'Memory': 2, 'Equip Slots': 0, 'HP': 590, 'SP': 77, 'Atk': 79, 'Def': 69, 'Int': 68, 'Spd': 95}


Okay, since I've got the data in the list of dictionaries like I want, let's move to the next step.

### 2 - 4: Saving the Data into .JSON, .CSV, .XLSX Format

In this process, I want to save the data I've been scraped into 3 format, which are .JSON, .CSV, and .XLSX. I'll be using 3 libraries for each format, let's do the .JSON format first.  

For the import part, I'll import all the library I need in one go, here are the details:  
 - JSON format: for .JSON format, I'm gonna use [json](https://docs.python.org/3/library/json.html) library.
 - CSV format: for .CSV format, I'm gonna use [csv](https://docs.python.org/3/library/csv.html) library.
 - XLSX format: for .XLSX format, I'm gonna use [xlsxwriter](https://xlsxwriter.readthedocs.io/) library.

In [17]:
import json
import csv
import xlsxwriter

Since I already have the data that I wanna write, I just need to specify the JSON file and passing the **_processedData_** variable into **_jsonFile_** and _et voilà!_ Writing to .JSON format is completed!

In [16]:
with open("digimon.json","w") as jsonFile:
    json.dump(processedData, jsonFile)
    
print(f"Success! The file '{jsonFile.name}' has been saved!")

Success! The file 'digimon.json' has been saved!


For the .CSV format, the process is quite simple too. Because the data that I wanna write is in list of dictionary, I can use DictWriter to write it. I just need to specify the keys which will be the fieldnames, and that's it. The DictWriter will do the rest.

In [19]:
with open("digimon.csv","w",newline="") as csvFile:
    writer = csv.DictWriter(csvFile, delimiter=";", fieldnames=list(processedData[0].keys()))
    writer.writeheader()
    writer.writerows(processedData)
    
print(f"Success! The file '{csvFile.name}' has been saved!")

Success! The file 'digimon.csv' has been saved!


For the .XLSX format, I'm gonna use for loop to write out the column header by using dictionary keys, and then I'll use nested for loop to input the table value using dictionary values. But before that, I need to specify the workbook and sheet using **xlsxwriter.Workbook** _(for workbook)_ and **workbook.add_worksheet** _(to add sheet in the workbook)_

In [24]:
workbook = xlsxwriter.Workbook("digimon.xlsx")
sheet = workbook.add_worksheet("Sheet1")

key = list(processedData[0].keys())
values = list(processedData[0].values())

for col, data in enumerate(key):
    sheet.write(0, col, data)

for col, item in enumerate(processedData,start = 1): #start at 1 because row 0 are used for column header
    for index, data in enumerate(item.values()):
        sheet.write(col, index, data)
        
workbook.close()

print(f"Success! The file '{workbook.filename}' has been saved!")

Success! The file 'digimon.xlsx' has been saved!


### 5. Storing Data in MySQL Database

In this process, I want to store the scraped data into MySQL Database. To do that, I need [mysql connector](https://dev.mysql.com/doc/connector-python/en/) library. So let's import it and move on to the next step.

In [1]:
from mysql import connector

To use mysql connector, I need to establish the connection to mysql database first before I can execute any mysql query with python. So I create _my_database_ variable which contains connect class and provide connect function with my connection parameter. After that, I create a variable called _cursor_ which contains cursor class. Cursor class instantiates objects that can execute operations such as SQL statements, I provide it with dictionary=True so the cursor will return it's query as dictionary format.

In [7]:
my_database = connector.connect(host='localhost', port=3306, user='root', passwd='password')

cursor = my_database.cursor(dictionary=True)

After I establish the connection and setting up the cursor, I can use cursor.execute to process MySQL query from python. Here, I'm creating a database called digiworld, set it as active database and creating a table called digimon.

In [8]:
cursor.execute("CREATE DATABASE digiworld")
cursor.execute("USE digiworld")
cursor.execute("CREATE TABLE digimon(id INT PRIMARY KEY NOT NULL AUTO_INCREMENT,imageLink VARCHAR(255), digimon VARCHAR(255), stage VARCHAR(255), digiType VARCHAR(255)," 
              "attribute VARCHAR(255),memory INT, equipSlots INT, hp INT, sp INT, atk INT, defense INT, intel INT, spd INT)")

The next step after creating database and creating table is inserting value into the table that I've just created. I used execute many so I don't need looping to process it. Before I used executemany, I need to provide the value to be executed so I create variable called _val_ which contains a list of dictionary value in processedData. After the cursor running the executemany I have to make the change happened in my database, so I used _database.commit()_ in order to do that.

In [27]:
sql = "INSERT INTO digimon VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
val = [tuple(x.values()) for x in processedData]
cursor.executemany(sql, val)

my_database.commit()

print(cursor.rowcount, "records inserted.")

341 records inserted.


### 6. Storing Data in MongoDB Database

The last process will be storing the scraped data into MongoDB database. In this part, I'll be using [pymongo](https://api.mongodb.com/python/current/) library to do that. So let's just import it first.

In [19]:
from pymongo import MongoClient

After I've imported the pymongo, I also need to establish the connection first like MySQL, the connection will be called _client_. After I establish the connection, I can create a database by using _client['digiworld']_ and create collection called _digimon_list_. The collection won't be shown until there is at least 1 record in it.

In [20]:
urldb = "mongodb://localhost:27017"
client = MongoClient(urldb)
database = client['digiworld']
collection = database['digimon_list']

To insert all the scraped data into collection in one go, I used _collection.insert_many_ and just passed the _processedData_ dictionary without the need to grab the keys or values.

In [28]:
x = collection.insert_many(processedData)

print(f'{len(x.inserted_ids)} records inserted.')

341 records inserted.
